# Julia
- IDE: VSCode (OpenSource)
- keine Klassenmethoden, stattdessen type check bei Funktionen
    ```julia
    function beispiel(n::int)
        return n+n
    end
    ```
- keine automatische type conversion
    -> Funktion würde error schmeißen wenn z.B. 1.0 übergeben wird
- Pkg: Julia's Package Manager
- Environment aufsetzen:
    * Package Manager aktivieren: ```using Pkg```
    * Environment aktivieren: ```Pkg.activate("")```
    * Package installieren: ```Pkg.add("PackageName")```
    * Dadurch werden automatisch eine Project Toml (enthält alle Packages, die man geadded hat) und eine Manifest Toml (enthält alle dependencies) erstellt
- Existierende Environment installieren:
    * ```Pkg.activate("")```
    * ```Pkg.instantiate()```

In [ ]:
using eDisGo_OPF
using PowerModels
using Ipopt
using JuMP
using JSON
using Gurobi

# PowerModels
## 1. [Network Data Format](https://lanl-ansi.github.io/PowerModels.jl/stable/network-data/)
- PowerModels arbeitet mit Dictionaries
- IO: Matpower .m-file, PTI .raw-file oder json
- eDisGo: Dictionary "pm" wird aus eDisGo-Objekt in ```edisgo.io.powermodels_io.to_powermodels()``` erzeugt und in ```edisgo.opf.powermodels_opf.pm_optimize()``` als json-string an Julia subproccess übergeben

In [ ]:
# Example
data = parse_file("./test/case3.m")
data

In [ ]:
data["bus"]["1"]

In [ ]:
print_summary(data) # funktioniert nicht für Multi-period Network Data

## 2. Optimierungsmodell erstellen
PowerModels Funktion 
```julia 
instantiate_model(data, model_type, build_method)
```
- data: Netzdaten im Network Data Format
- model_type: verwendete Modellformulierung (z.B. BIM, BFM, linearisiert, SOC, ...). Modelltypen findet man [hier](https://lanl-ansi.github.io/PowerModels.jl/stable/formulation-details/)
- build_method(): enthält die Funktionen zur Variablen-, Constraint- und Objective Function-Erzeugung für das Modell

In [ ]:
# DCPPowerModel: Linearized 'DC' power flow Model with polar voltage variables.
pm = instantiate_model(data, DCPPowerModel, build_opf)
pm.model

In [ ]:
print(pm.model)

## 3. Optimierungsmodell lösen
PowerModels Funktion 
```julia 
optimize_model!(pm; optimizer=nothing)
```
- pm: PowerModels Optimierungsmodell
- optimizer: Solver kann hier gesetzt werden (falls nicht wird default verwendet)

In [ ]:
result_ipopt = optimize_model!(pm, optimizer=Ipopt.Optimizer);

In [ ]:
pm = instantiate_model(data, DCPPowerModel, build_opf)
result_gurobi = optimize_model!(pm, optimizer=Gurobi.Optimizer);

## Alternative zu 2. + 3.
Statt 2. und 3. einzeln auszuführen, kann das Modell auch direkt gelöst werden mit der PowerModels Funktion 
```julia
solve_opf(data, model_type; optimizer)
```

In [ ]:
solve_opf(data, DCPPowerModel, Gurobi.Optimizer);

## 4. [PowerModels Result Data Format](https://lanl-ansi.github.io/PowerModels.jl/stable/result-data/)

In [ ]:
print_summary(result_ipopt["solution"])

In [ ]:
print_summary(result_gurobi["solution"])

## 5. Ergebnisse auf die ursprünglichen Daten schreiben
Mit der PowerModels Funktion
```julia
update_data!(data, result["solution"])
```

In [ ]:
data["gen"]["1"]

In [ ]:
update_data!(data, result_gurobi["solution"])

In [ ]:
data["gen"]["1"]

## 6. Solvereinstellungen

In [ ]:
const ipopt = optimizer_with_attributes(Ipopt.Optimizer, MOI.Silent() => false, "sb" => "yes");
const gurobi = optimizer_with_attributes(Gurobi.Optimizer, MOI.Silent() => false, "Presolve" => 1, 
                                        "BarQCPConvTol" => 1e-5, "BarConvTol" => 1e-6);

* Beispiel mit eDisGo json file (save_edisgo_to_json()) -> zum Debuggen + um sich die Datentransformation zu sparen
* neues python jupyter notebook in dem Vorgehen aus eDisGo heraus durchgeführt wird!

## 7. Beispiel eDisGo 
* Zum Debuggen in Julia kann eDisGo Netzwerk im PowerModels Network Data Format als json gespeichert werden mit eDisGo Funktion 
```python
edisgo.save_edisgo_to_json()
```
### 7.1. Einlesen der Daten und Erstellung eines Multinetworks
* Einlesen der Daten mit PowerModels ```parse_file(path)``` Funktion
* Für Multi-Period-Optimierung muss aus den eDisGo-Daten ein "multi network" gemacht werden mit der PowerModels Funktion
```julia
make_multinetwork(data)
```


In [ ]:
data_edisgo = parse_file("./test/ding0_1_t_24.json");
print_summary(data_edisgo)
data_edisgo_mn = make_multinetwork(data_edisgo);

### 7.2. Lösung des OPFs
Für das eDisGo-OPF Problem inklusiver aller (wahlweise auch einer Auswahl an) Flexibilitäten wird die eDisGo_OPF Funktion
```julia
solve_mn_opf_bf_flex(data_mn, model_type, optimizer)
```
verwendet. 
Zum Lösen des eDisGo-OPFs stehen zwei Modellansätze zur Verfügung: konvexe (SOC) Optimierung und exakte, nicht-konvexe Optimierung. Bei der konvexen Optimierung wird die nicht-konvexe Leistungsgleichung $P^{2} + Q^2 = I^2 \cdot V^2$ relaxiert zu $P^{2} + Q^2 \leq I^2 \cdot V^2$.
#### 7.2.1. Konvexe Optimierung
- model_type: SOCBFPowerModelEdisgo
- optimizer: Gurobi




In [ ]:
result_soc, pm = eDisGo_OPF.solve_mn_opf_bf_flex(data_edisgo_mn, SOCBFPowerModelEdisgo, gurobi);

In [ ]:
print_summary(result_soc)

#### Ergebnisse der einzelnen Zeitschritte ausgeben
```julia
result["solution"]["nw"][t]
```
mit t: Zeitschritt als String

In [ ]:
print_summary(result_soc["solution"]["nw"]["1"])

Da es sich bei der konvexen Optimierung um eine SOC-Relaxation des urspünglichen Problems handelt, muss überprüft werden, ob die Exaktheit der relaxierten Gleichung erfüllt ist, also ob $P^{2} + Q^2 = I^2 \cdot V^2$ gilt. Dafür kann die eDisGo_OPF Funktion
```julia
soc_tight, soc_dict = check_SOC_equality(result_soc, data_edisgo)
```
verwendet werden. Sie gibt zurück, ob die SOC Lösung exakt ist (soc_tight: boolean) und zudem ein Dictionary mit den Ergebnissen der Berechnung $P^{2} + Q^2 - I^2 \cdot V^2$ für alle Zeitschritte und Branches, in denen die SOC-Lösung nicht exakt ist (soc_dict: dict).

In [ ]:
soc_tight, soc_dict = eDisGo_OPF.check_SOC_equality(result_soc, data_edisgo);
soc_dict

#### 7.2.2 Nicht-konvexe Optimierung
- model_type: NCBFPowerModelEdisgo
- optimizer: Ipopt

In [ ]:
result_nc = eDisGo_OPF.solve_mn_opf_bf_flex(data_edisgo_mn, NCBFPowerModelEdisgo, ipopt);

In [ ]:
print_summary(result_nc)

#### Nicht-konvexe Optimierung warm-starten
Es besteht die Möglichkeit, die nicht-konvexe Optimierung mit der SOC-Lösung warm zu starten. Dazu die Ergebnisse der SOC-Lösung in die eDisGo-Netzdaten (data_edisgo_mn) integrieren und die Startwerde mit der eDisGo_OPF Funktion ```set_ac_bf_start_values!(data_edisgo_mn["nw"]["1"])``` setzen. Danach die nicht-konvexe Optimierung wie gewohnt starten.

```julia
update_data!(data_edisgo_mn, result_soc["solution"])
set_ac_bf_start_values!(data_edisgo_mn["nw"]["1"])
solve_mn_opf_bf_flex(data_edisgo_mn, NCBFPowerModelEdisgo, ipopt)
```

In [ ]:
update_data!(data_edisgo_mn, result_soc["solution"])
eDisGo_OPF.set_ac_bf_start_values!(data_edisgo_mn["nw"]["1"])
eDisGo_OPF.solve_mn_opf_bf_flex(data_edisgo_mn, NCBFPowerModelEdisgo, ipopt);